## Initial Imports

In [2]:
from scipy.io import arff
import pandas as pd
import numpy as np
import pandas as pd
import plotly.express as px
import sys
import argparse
import os
from filelock import FileLock
num_gpus = 1 ## set to more if you have a machine with more gpus
num_cpus = 1 ## set to more if you have a machine with more CPUs - recommend at least 1 per GPU +1

## Base directory must include RayPackages folder
base_directory = "/project/DSone/jaj4zcf/UCI_Time_Series/"


## Import Sample Multivariate Archive
- Must set data_directory to location of multivariate archive to work.
- Skip if you want to use your own dataset
- Make sure to normalize multivariate data used for best results.

In [3]:
data_directory = '/project/DSone/jaj4zcf/UCI_Time_Series/Multivariate_arff/'
colnames=pd.read_csv(data_directory+'DataDimensions.csv', names=[0]).reset_index().iloc[0].values
data = pd.read_csv(data_directory+'DataDimensions.csv', names=[0]).reset_index().iloc[1:]
data.columns = colnames
data.loc[1].Problem
data.loc[1].NumDimensions
data

,Problem,TrainSize,TestSize,NumDimensions,SeriesLength,NumClasses,Normalised,Padded,MissingValues,ClassCounts
1,ArticularyWordRecognition,275,300,9,144,25,true,false,false,11
2,AtrialFibrillation,15,15,2,640,3,false,false,false,5
3,BasicMotions,40,40,6,100,4,false,false,false,10
4,CharacterTrajectories,1422,1436,3,182,20,false,false,false,85
5,Cricket,108,72,6,1197,12,true,false,false,9
6,DuckDuckGeese,50,50,1345,270,5,false,false,false,10
7,EigenWorms,128,131,6,17984,5,false,false,false,55
8,Epilepsy,137,138,3,206,4,false,false,false,34
9,EthanolConcentration,261,263,3,1751,4,false,false,false,65
10,ERing,30,270,4,65,6,false,false,false,5


In [3]:
## Function to Load Multivariate Archive
def load_multivariate_dataset(directory='/project/DSone/jaj4zcf/UCI_Time_Series/Multivariate_arff/', name='NATOPS'):
    if name == 'Phoneme':
        name = 'PhonemeSpectra'
    df = arff.loadarff(directory+name+'/'+name +'_TRAIN.arff' )
    #df = pd.DataFrame(df[0])
    df = list(df)
    dataset=df[0]
    len(list(dataset))
    class_vals = []
    data_vals = []
    for sample in dataset:
        data_vals.append(np.array(pd.DataFrame(sample[0]).T))
        try:
            class_vals.append(int(float(sample[1].decode("utf-8"))))
        except:
            class_vals.append(sample[1].decode("utf-8"))
    train_data_x=np.array(data_vals)
    train_data_y=np.array(class_vals)
    df = arff.loadarff(directory+name+'/'+name +'_TEST.arff' )
    #df = pd.DataFrame(df[0])
    df = list(df)
    dataset=df[0]
    len(list(dataset))
    class_vals = []
    data_vals = []
    for sample in dataset:
        data_vals.append(np.array(pd.DataFrame(sample[0]).T))
        try:
            class_vals.append(int(float(sample[1].decode("utf-8"))))
        except:
            class_vals.append(sample[1].decode("utf-8"))
    test_data_x=np.array(data_vals)
    test_data_y=np.array(class_vals)
    #np.int(list(sample)[1].decode("utf-8"))
    std_ = train_data_x.std(axis=1, keepdims=True)
    std_[std_ == 0] = 1.0
    train_data_x = (train_data_x - train_data_x.mean(axis=1, keepdims=True)) / std_
    std_ = test_data_x.std(axis=1, keepdims=True)
    std_[std_ == 0] = 1.0
    test_data_x = (test_data_x - test_data_x.mean(axis=1, keepdims=True)) / std_
    return train_data_x, train_data_y, test_data_x, test_data_y

multivariate=['ArticularyWordRecognition', 'AtrialFibrillation', 'BasicMotions', 'Cricket','DuckDuckGeese',
'EigenWorms','Epilepsy','EthanolConcentration','ERing','FaceDetection','FingerMovements','HandMovementDirection',
'Handwriting','Heartbeat','Libras','LSST','MotorImagery','NATOPS','PenDigits','PEMS-SF','PhonemeSpectra','RacketSports',
'SelfRegulationSCP1','SelfRegulationSCP2','StandWalkJump','UWaveGestureLibrary']

#times = []

#for name in multivariate:
#    time1 = time.time()
#    x_train,y_train,x_test,y_test=load_multivariate_datset(name=name)
#    time1 = time.time()-time1
#    times.append(time1)
#    print(times)

## Sorted order for multivariate data sets by time to import 

multiTimeOrder = ['ERing', 'BasicMotions', 'AtrialFibrillation', 'RacketSports',
       'Libras', 'NATOPS', 'Epilepsy', 'StandWalkJump', 'Handwriting',
       'FingerMovements', 'UWaveGestureLibrary',
       'ArticularyWordRecognition', 'HandMovementDirection', 'Cricket',
       'LSST', 'SelfRegulationSCP2', 'SelfRegulationSCP1', 'Heartbeat',
       'PenDigits', 'EthanolConcentration', 'DuckDuckGeese',
       'PhonemeSpectra', 'PEMS-SF', 'MotorImagery', 'FaceDetection',
       'EigenWorms']

## Import Augmentations 

In [4]:
## Imports augmentations from multiv_augs_PBA_raytune.py
## Ensure that 'RayPackages' folder is on your path 

sys.path.insert(1, '/project/DSone/jaj4zcf/UCI_Time_Series/')


from RayPackages import multiv_augs_PBA_raytune as augs_PBA_raytune

augs_func_list=[augs_PBA_raytune.jitter, augs_PBA_raytune.scaling, 
 augs_PBA_raytune.rotation, augs_PBA_raytune.permutation, 
 augs_PBA_raytune.time_warp,augs_PBA_raytune.window_slice,
 augs_PBA_raytune.crop, augs_PBA_raytune.magnitude_warp,
 augs_PBA_raytune.window_warp, augs_PBA_raytune.axes_rotation]


### Build Policies and Augment Function
## Test Code to Implement Random Series of Augmentation/magnitude pairs

augs_func_list=[augs_PBA_raytune.jitter, augs_PBA_raytune.scaling, 
 augs_PBA_raytune.rotation, augs_PBA_raytune.permutation, 
 augs_PBA_raytune.time_warp,augs_PBA_raytune.window_slice,
 augs_PBA_raytune.crop, augs_PBA_raytune.magnitude_warp,
 augs_PBA_raytune.window_warp,augs_PBA_raytune.axes_rotation]


aug_groups = {}
aug_groups['noise'] = [augs_PBA_raytune.jitter]
aug_groups['rotation'] =[augs_PBA_raytune.rotation]
aug_groups['permutation'] =[augs_PBA_raytune.permutation]
aug_groups['magnitude'] = [augs_PBA_raytune.magnitude_warp]
aug_groups['scaling'] = [augs_PBA_raytune.scaling]
aug_groups['time_warp'] = [augs_PBA_raytune.time_warp]
aug_groups['window_warp'] = [augs_PBA_raytune.window_warp]
aug_groups['crop'] = [augs_PBA_raytune.crop]
aug_groups['window_slice'] = [augs_PBA_raytune.window_slice]
aug_groups['axes_rotation'] = [augs_PBA_raytune.axes_rotation]

num_aug_groups = len(aug_groups.keys())

## Will be used to create policy  
setting_cols = ['config/aug_' + k +'_prob' for k in aug_groups.keys()]

setting_cols



['config/aug_noise_prob',
 'config/aug_rotation_prob',
 'config/aug_permutation_prob',
 'config/aug_magnitude_prob',
 'config/aug_scaling_prob',
 'config/aug_time_warp_prob',
 'config/aug_window_warp_prob',
 'config/aug_crop_prob',
 'config/aug_window_slice_prob',
 'config/aug_axes_rotation_prob']

In [5]:
from tensorflow.keras.models import load_model
import ray
from ray import tune
from ray.tune.schedulers import PopulationBasedTraining, AsyncHyperBandScheduler,PopulationBasedTrainingReplay
from ray.tune.integration.keras import TuneReportCallback
import sklearn
import RayPackages
from RayPackages import multiv_augs_PBA_raytune as augs_PBA_raytune
from RayPackages.utils import read_dataset as read_dataset
from RayPackages.utils import transform_labels, calculate_metrics
from RayPackages.stoppers import CMaximumIterationStopper, EarlyStopping
import numpy as np
import pandas as pd
from tensorflow import keras
from sklearn import model_selection
from tensorflow.keras import backend as K
from typing import Dict, Optional
import time
from collections import defaultdict, deque
import numpy as np
import random
import string
import json
from ray import logger
from ray.util.annotations import PublicAPI   
from ray.tune import JupyterNotebookReporter
from sklearn.preprocessing import LabelEncoder
from ray.tune.stopper import CombinedStopper, MaximumIterationStopper, TrialPlateauStopper, TimeoutStopper
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import StratifiedShuffleSplit
import time
from RayPackages.model_build import build_model, DataGeneratorClassifyMultivariate, augment
start_time = time.time()

In [6]:


## Training

class train_set_TS(tune.Trainable):
    def build_model(self):
        model = build_model(input_shape=self.input_shape,nb_classes=self.nb_classes, model_type=self.config['model'],dense_layer=config['dense_layer'],num_kernel_sizes =config['num_kernel_sizes'], kernel_size =config['kernel_size'], 
                            dropout=self.config['dropout'], n_feature_maps = self.config['filters'], depth = self.config['depth'] )
        print('Model Initialized')
        if len(self.config['initial_model_locs'])>0:
            print('LOADING PRETRAINED WEIGHTS')
            path = np.random.choice(self.config['initial_model_locs'])
            model.load_weights(path)
            print('LOADED PRETRAINED WEIGHTS')

        return model

    def setup(self, config):
        self.config=config
        self.monitor = config['monitor'] 
        self.use_val = config['use_val']
        self.mini_batch = config['mini_batch_split']
                
        if self.config['grid_search']:
                for i in range(len(self.config['grid_search_space'])):
                    self.config[setting_cols[i][7:]]=self.config['grid_search_space'][i]
        #print(config)
        
        with FileLock(os.path.expanduser("~/.tune.lock")):
            
            if hasattr(self, 'x_train')==False:
                if True:
                    x_train,y_train,x_test,y_test=load_multivariate_dataset(directory='/project/DSone/jaj4zcf/UCI_Time_Series/Multivariate_arff/', name=config['dataset_name'])
                    self.x_train = x_train
                    self.x_test = x_test
                    self.y_train= y_train
                    self.y_test= y_test
                else:
                    dataset=read_dataset(root_dir='/project/DSone/jaj4zcf/UCI_Time_Series/UCR_TS_Archive_2015/' , archive_name='UCRArchive_2015', dataset_name=config['dataset_name'])
                    self.x_train = dataset[config['dataset_name']][0]
                    self.y_train = dataset[config['dataset_name']][1]
                    self.x_test = dataset[config['dataset_name']][2]
                    self.y_test = dataset[config['dataset_name']][3]

                if self.use_val == True:
                    split = self.config['split']
                    sss = StratifiedShuffleSplit(n_splits=config['n_splits'], test_size=1/config['n_splits'], random_state=1)
                    sss.get_n_splits(self.x_train,self.y_train)
                    splits=list(sss.split(self.x_train,self.y_train))
                    self.x_train_s=self.x_train[splits[split][0]]
                    self.x_val_s=self.x_train[splits[split][1]]
                    self.y_train_s=self.y_train[splits[split][0]]
                    self.y_val_s=self.y_train[splits[split][1]]
                self.nb_classes = len(np.unique(np.concatenate((self.y_train, self.y_test), axis=0)))

                # transform the labels from integers to one hot vectors
                enc = sklearn.preprocessing.OneHotEncoder(categories='auto')
                enc.fit(np.concatenate((self.y_train, self.y_test), axis=0).reshape(-1, 1))
                self.y_train = enc.transform(self.y_train.reshape(-1, 1)).toarray()  
                if self.use_val == True:
                    self.y_train_s = enc.transform(self.y_train_s.reshape(-1, 1)).toarray()
                    self.y_val_s = enc.transform(self.y_val_s.reshape(-1, 1)).toarray()  
                self.y_test = enc.transform(self.y_test.reshape(-1, 1)).toarray()
                # save orignal y because later we will use binary
                self.y_true = np.argmax(self.y_test, axis=1)

            if len(self.x_train.shape) == 2:  # if univariate
            #    # add a dimension to make it multivariate with one dimension 
                self.x_train = self.x_train.reshape((self.x_train.shape[0], self.x_train.shape[1], 1))
                self.x_test = self.x_test.reshape((self.x_test.shape[0], self.x_test.shape[1], 1))
                if self.use_val == True:
                    self.x_train_s = self.x_train_s.reshape((self.x_train_s.shape[0], self.x_train_s.shape[1], 1))
                    self.x_val_s = self.x_val_s.reshape((self.x_val_s.shape[0], self.x_val_s.shape[1], 1))

        self.input_shape = self.x_train.shape[1:]
     
        #to ensure the same batch_size as when training on full data
        self.batch_size = int(min(config['batch_size'],self.x_train.shape[0]/self.mini_batch)) #int(min(self.x_train.shape[0] / 10, config['batch_size']))
        if self.use_val == True:
            #self.batch_size = int(min(config['batch_size'],self.x_train_s.shape[0]/self.mini_batch)) #int(min(self.x_train.shape[0] / 10, config['batch_size']))
            self.dgen_class=DataGeneratorClassifyMultivariate(self.x_train_s,self.y_train_s,self.x_val_s,self.y_val_s,aug_groups,augment=self.config['data_augment'],config=self.config,batch_size=self.batch_size)
        else:
            self.dgen_class=DataGeneratorClassifyMultivariate(self.x_train,self.y_train,self.x_test,self.y_test,aug_groups,augment=self.config['data_augment'],config=self.config,batch_size=self.batch_size)
        

    def step(self):
        # train
        #print('current LR')
        #print(self.config['lr'])
        if self.iteration == 0:
            model = self.build_model()
            self.lr = config['lr']
            model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr=config["lr"]),
                          metrics=['accuracy'])
            self.model = model
        elif hasattr(self, 'model') == False:
            model = self.build_model()
            self.lr = config['lr']
            model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr=config["lr"]),
                          metrics=['accuracy'])
            self.model = model
        
        if self.use_val == True:
            history = self.model.fit(
                self.dgen_class,
                epochs=self.config.get("epochs", 1),
                validation_data=(self.x_val_s, self.y_val_s),
                verbose=0)
            if self.monitor == True:
                res = self.model.evaluate(self.x_test,self.y_test,verbose=0) #np.mean(history.history['loss'][-1:])
            else:
                res = (0,0)
            res1=self.model.evaluate(self.x_train_s,self.y_train_s,verbose=0)
            self.reduce_lr_on_plateau(res1[0])
            #self.config['current_loss'] = np.mean(history.history['val_loss'])
            #self.reduce_lr_on_plateau()
            #if self.config['current_loss']<self.config['best_loss']:
            #    self.config['best_loss'] = self.config['current_loss']
            val_loss = -np.mean(history.history['val_loss'][-1:])
            val_acc = np.mean(history.history['val_accuracy'][-1:])
            test_loss = res[0]
            test_acc = res[1]
            loss = -res1[0] #evaulate loss on NON-augmented data
            return {"mean_loss": loss, "test_acc": test_acc,"val_acc": val_acc,"val_loss": val_loss}

        else:
            if self.monitor == True:
                history = self.model.fit(
                    self.dgen_class,
                    epochs=self.config.get("epochs", 1),
                    validation_data=(self.x_train, self.y_train),
                    verbose=0)
                res = self.model.evaluate(self.x_test,self.y_test,verbose=0)
                self.reduce_lr_on_plateau(res[0])
                #np.mean(history.history['loss'][-1:])
                train_loss = -np.mean(history.history['val_loss'][-1:])
                train_loss_avg = -np.mean(history.history['val_loss'][-5:]) 
                train_acc = np.mean(history.history['val_accuracy'][-1:]) 
                test_loss = -res[0]
                test_acc = res[1]
                acc = np.mean(history.history['accuracy'][-1:])
                return {"mean_loss": train_loss,"mean_loss_avg": train_loss_avg, "test_acc": test_acc,"train_acc": train_acc, 'lr':self.lr, 'best_loss':self.best_loss,'loss_count':self.loss_count}
            else:
                history = self.model.fit(
                    self.dgen_class,
                    epochs=self.config.get("epochs", 1),
                    validation_data=(self.x_train, self.y_train),
                    verbose=0)
                train_loss = -np.mean(history.history['val_loss'][-1:])
                train_loss_avg = -np.mean(history.history['val_loss'][-5:]) 
                self.reduce_lr_on_plateau(-train_loss)
                train_acc = np.mean(history.history['val_accuracy'][-1:])
                return {"mean_loss": train_loss,"mean_loss_avg": train_loss_avg, "test_acc": 0,"train_acc": train_acc}
    
    def reduce_lr_on_plateau(self, current_loss):
        min_delta = .001
        min_lr = .00005 #0.0001
        patience = 35
        
        if self._iteration == 0:
            self.best_loss = 9e9
            self.loss_count = 0

        if (self.best_loss - current_loss) > min_delta:
            self.loss_count = 0
            self.best_loss = current_loss
            
        else:
            self.loss_count = self.loss_count + 1
            if self.loss_count > patience:
                print('Plateau - reducing LR')
                self.lr = max(self.lr * .667, min_lr)
                K.set_value(self.model.optimizer.learning_rate, self.lr)
                print(self.lr)
                self.loss_count = 0
                
    def save_checkpoint(self, checkpoint_dir):
        file_path = checkpoint_dir 
        #print(file_path)
        self.model.save_weights(file_path + "/model.h5")
        #print('saved')
        details = {'lr':self.lr, 'best_loss':self.best_loss, 'loss_count':self.loss_count}
        with open(file_path + '/lr.txt', 'w') as convert_file:
            convert_file.write(json.dumps(details))
        return file_path                
    
    def load_checkpoint(self, path):
        # See https://stackoverflow.com/a/42763323
        print('loading_checkpoint')
        print('GEN INITIALIZED!!')

        with open(path + '/lr.txt', 'r') as convert_file:
            details=json.load(convert_file)
        self.lr = details['lr']
        self.best_loss = details['best_loss']
        self.loss_count = details['loss_count']
        
        self.model.load_weights(path + "/model.h5")
        if self.use_val == True:
            #self.batch_size = int(min(config['batch_size'],self.x_train_s.shape[0]/self.mini_batch)) #int(min(self.x_train.shape[0] / 10, config['batch_size']))
            self.dgen_class=DataGeneratorClassifyMultivariate(self.x_train_s,self.y_train_s,self.x_val_s,self.y_val_s,aug_groups,augment=self.config['data_augment'],config=self.config,batch_size=self.batch_size)
        else:
            self.dgen_class=DataGeneratorClassifyMultivariate(self.x_train,self.y_train,self.x_test,self.y_test,aug_groups,augment=self.config['data_augment'],config=self.config,batch_size=self.batch_size)        
        K.set_value(self.model.optimizer.learning_rate, self.lr)
        #print('model loaded')
    
    def reset(self, new_config, logger_creator=None):
              
        trial_info = new_config.pop(TRIAL_INFO, None)
        print(trial_info)
        if trial_info:
            self._trial_info = trial_info

        self._result_logger.flush()
        self._result_logger.close()

        if logger_creator:
            logger.debug("Logger reset.")
            self._create_logger(new_config.copy(), logger_creator)
        else:
            logger.debug(
                "Did not reset logger. Got: "
                f"trainable.reset(logger_creator={logger_creator})."
            )

        stdout_file = new_config.pop(STDOUT_FILE, None)
        stderr_file = new_config.pop(STDERR_FILE, None)

        self._close_logfiles()
        self._open_logfiles(stdout_file, stderr_file)
        print('self.reset')
        self.setup(new_config)

        # Reset attributes. Will be overwritten by `restore` if a checkpoint
        # is provided.
        self._iteration = 0
        self._time_total = 0.0
        self._timesteps_total = None
        self._episodes_total = None
        self._time_since_restore = 0.0
        self._timesteps_since_restore = 0
        self._iterations_since_restore = 0
        self._restored = False
        return True
    
    def reset_config(self, confignew):
        print('resetting_config!!!!!')
        self.config=confignew
        if self.use_val == True:
            #self.batch_size = int(min(config['batch_size'],self.x_train_s.shape[0]/self.mini_batch)) #int(min(self.x_train.shape[0] / 10, config['batch_size']))
            self.dgen_class=DataGeneratorClassifyMultivariate(self.x_train_s,self.y_train_s,self.x_val_s,self.y_val_s,aug_groups,augment=self.config['data_augment'],config=self.config,batch_size=self.batch_size)
        else:
            self.dgen_class=DataGeneratorClassifyMultivariate(self.x_train,self.y_train,self.x_test,self.y_test,aug_groups,augment=self.config['data_augment'],config=self.config,batch_size=self.batch_size)   
        #K.set_value(self.model.optimizer.learning_rate, self.lr)
        return True


from ray.tune.result import (TRIAL_INFO, STDOUT_FILE,STDERR_FILE)

## Set a maximum training time if desired, currently not used but can be added to combined stopper.
## Using TimeoutStopper - see ray.tune documentation

timeout = 60 * 60 * 12 - 120   
 

def explore(config_in):
    choice=np.linspace(0,1,11)
    for param in config_in:
        if param[0:3]=='aug':
            if np.random.random()<.5:
                if np.random.random()<.2:
                    temp = config_in[param] ## keep same or randomly perturb
                    # temp = np.random.choice(choice) ## randomly perturb OR
                    ## modify and clip probabilities
                else:
                    if np.random.random()<.5:
                        temp=config_in[param]+.1
                    else:
                        temp=config_in[param]-.1

                config_in[param]= np.clip(temp,0,1) 
    return config_in
   
    
    
def tune_PBT(config):   # tune an initial 4 models on 100% of the data for a short number of epochs before we start validation splits. 
    pbt = PopulationBasedTraining(
        time_attr="training_iteration",
        perturbation_interval= config['Pert_Interval'],
        quantile_fraction = 0.25,
        burn_in_period = 75,
        custom_explore_fn=explore)
    
    analysis = tune.run(
        train_set_TS,
        name='PBT_'+config['dataset_name'],
        #progress_reporter=ray.tune.CLIReporter(max_report_frequency = 120),
        progress_reporter=JupyterNotebookReporter(overwrite=True,max_progress_rows=30),
        resources_per_trial={
            "cpu": 1,
            "gpu": 1
        },
        scheduler=pbt,
        checkpoint_at_end=True, checkpoint_freq=config['check_freq'], keep_checkpoints_num=3,
        checkpoint_score_attr='mean_loss',
        stop= CombinedStopper(CMaximumIterationStopper(max_iter=config['training_iterations']),EarlyStopping(metric='mean_loss', metric_threshold=0.001, mode = 'max', patience=config['patience'], warmup=config['warmup'])),
        config=config,
        metric='mean_loss',
        mode='max',
        reuse_actors=True,
        num_samples=config['samples'],
        local_dir = "/project/DSone/jaj4zcf/UCI_Time_Series/"+str(sys.argv[2])+'/'
        )
    return analysis    

In [8]:
config={}

summary = pd.read_csv('/project/DSone/jaj4zcf/UCI_Time_Series/DataSummary.csv')
summary = summary[np.array(summary.Length)!='Vary']
summary.Length=np.array(summary.Length).astype(int)
summary['Depth']=np.clip((summary.Length/40),6,8).round(0).astype(int)
summary=summary.set_index('Name')

print("1:dataset, 2:Folder, 3:PBT Iteration, 4: Sample Size, 5: pert Interval, 6: Patience, 7: Warmup, 8 Depth, 9: Filters  ")

sys.argv[1:] = ['Heartbeat', 'GridSearch8', 1500, 20, 20, 75,50, 4, 16,True]

## Global Choices
splits = 2
dataset = sys.argv[1]
config['dataset_name']=dataset

config['data_augment']= True
config['monitor'] = True  #observe test loss during training
# model
config['model']='inceptionFCNMulti2'
config['depth'] = int(sys.argv[8])
config['filters']= int(sys.argv[9])
config['dropout']= 0 
config['num_kernel_sizes']= 3 
config['dense_layer']= 0 
config['kernel_size']= 41 
config['Pert_Interval'] = int(sys.argv[5])
#Training
config['optimizer'] = 'ADAM'
config['lr']=.001
#config['momentum']=.0005 # Only if using not ADAM LR
config['batch_size']=16
config['mini_batch_split']=10
## Model Variables



## For each training run

#Training Parameters Stuff

config['patience'] = int(sys.argv[6])
config['split']=0
splits = 3
config['n_splits'] = splits
config['grid_search'] = False
config['epochs']= 1 # TF training Epochs per ray trainable step.
config['samples'] = int(sys.argv[4]) #vs 12?
config['training_iterations'] = int(sys.argv[3])
config['check_freq'] = 1
config['warmup'] = int(sys.argv[7])
      
## Initial Training With All Data
config['use_val'] = False
config['initial_model_locs']=[]
zeros=np.zeros(num_aug_groups)
search_space = np.vstack([zeros])
search_space = list(search_space)
config['grid_search_space']=zeros
config['split']=0 #tune.grid_search([0,1,2,3])
def add_aug_policy_train(config):
    for aug in aug_groups.keys():
        config['aug_'+aug+'_prob']=tune.choice([0])
add_aug_policy_train(config)
config['choice_probs'] = ([0,1],[.2,.8])

ray.shutdown()
ray.init(runtime_env={"py_modules": ['/project/DSone/jaj4zcf/UCI_Time_Series/RayPackages']}, num_cpus=num_cpus,num_gpus=num_gpus)

1:dataset, 2:Folder, 3:PBT Iteration, 4: Sample Size, 5: pert Interval, 6: Patience, 7: Warmup, 8 Depth, 9: Filters  


2022-06-22 11:34:27,911	INFO services.py:1462 -- View the Ray dashboard at http://127.0.0.1:8265
2022-06-22 11:34:29,398	INFO packaging.py:388 -- Creating a file package for local directory '/project/DSone/jaj4zcf/UCI_Time_Series/RayPackages'.
2022-06-22 11:34:29,420	INFO packaging.py:241 -- Pushing file package 'gcs://_ray_pkg_e7e8498b9d50a87e.zip' (0.24MiB) to Ray cluster...
2022-06-22 11:34:29,423	INFO packaging.py:243 -- Successfully pushed file package 'gcs://_ray_pkg_e7e8498b9d50a87e.zip'.


RayContext(dashboard_url='127.0.0.1:8265', python_version='3.7.12', ray_version='1.12.0', ray_commit='f18fc31c7562990955556899090f8e8656b48d2d', address_info={'node_ip_address': '10.155.195.126', 'raylet_ip_address': '10.155.195.126', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-06-22_11-34-25_156653_16222/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-06-22_11-34-25_156653_16222/sockets/raylet', 'webui_url': '127.0.0.1:8265', 'session_dir': '/tmp/ray/session_2022-06-22_11-34-25_156653_16222', 'metrics_export_port': 61470, 'gcs_address': '10.155.195.126:59394', 'address': '10.155.195.126:59394', 'node_id': '9fd88741c936927560485a573f93e5283288cd64d758f6dd5e6284ce'})

In [9]:
analysis_initial=tune_PBT(config=config)
trial_dfs=analysis_initial.fetch_trial_dataframes()
trial_dfs=list(trial_dfs.items())

Trial name,status,loc,aug_axes_rotation_prob,aug_crop_prob,aug_magnitude_prob,aug_noise_prob,aug_permutation_prob,aug_rotation_prob,aug_scaling_prob,aug_time_warp_prob,aug_window_slice_prob,aug_window_warp_prob
train_set_TS_d4d47_00000,RUNNING,10.155.195.126:17060,0,0,0,0,0,0,0,0,0,0
train_set_TS_d4d47_00001,PENDING,,0,0,0,0,0,0,0,0,0,0
train_set_TS_d4d47_00002,PENDING,,0,0,0,0,0,0,0,0,0,0
train_set_TS_d4d47_00003,PENDING,,0,0,0,0,0,0,0,0,0,0
train_set_TS_d4d47_00004,PENDING,,0,0,0,0,0,0,0,0,0,0
train_set_TS_d4d47_00005,PENDING,,0,0,0,0,0,0,0,0,0,0
train_set_TS_d4d47_00006,PENDING,,0,0,0,0,0,0,0,0,0,0
train_set_TS_d4d47_00007,PENDING,,0,0,0,0,0,0,0,0,0,0
train_set_TS_d4d47_00008,PENDING,,0,0,0,0,0,0,0,0,0,0
train_set_TS_d4d47_00009,PENDING,,0,0,0,0,0,0,0,0,0,0


2022-06-22 11:37:02,453	WARNING tune.py:651 -- SIGINT received (e.g. via Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C one more time (or send SIGINT/SIGKILL/SIGTERM) to skip. 
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-e96ec720a144>", line 1, in <module>
    analysis_initial=tune_PBT(config=config)
  File "<ipython-input-6-59ffb6224fab>", line 310, in tune_PBT
    local_dir = "/project/DSone/jaj4zcf/UCI_Time_Series/"+str(sys.argv[2])+'/'
  File "/home/jaj4zcf/.local/lib/python3.7/site-packages/ray/tune/tune.py", line 672, in run
    runner.step()
  File "/home/jaj4zcf/.local/lib/python3.7/site-packages/ray/tune/trial_runner.py", line 767, in step
    self._wait_and_handle_event(next_trial)
  File "/home/jaj4zcf/.local/lib/python3.7/site-packages/ray/tune/trial_runner.py", line 716, in _wait_and_handle_event
    self._live_trials, next_trial is not None
  File "/home/jaj4zcf/.local/lib/python3.7/site-packages/ray/tune/ray_trial_executor.py", line 857, in get_next_executor_event
    futu

TypeError: object of type 'NoneType' has no len()

## Get Predictions and Evaluate Ensemble Result 
Saves results in results.txt 

In [7]:
from ray.tune import ExperimentAnalysis
#### can be used to import an old training result
#analysis = ExperimentAnalysis("/project/DSone/jaj4zcf/UCI_Time_Series/multivariate/PBT_EigenWorms/experiment_state-2022-05-16_00-10-52.json")
#trial_dfs=analysis.fetch_trial_dataframes()
#trial_dfs=list(trial_dfs.items()) #analysis = ExperimentAnalysis("/project/DSone/jaj4zcf/UCI_Time_Series/GridSearch8/gridsearchArrowHead/experiment_state-2022-04-14_12-45-56.json")
#### Must Set Dataset Name
# config['dataset_name']='EigenWorms'


df = 0
for dfr in trial_dfs:
    try:
        row= dfr[1].sort_values(by=['mean_loss','training_iteration'], ascending=[True,False]).reset_index().iloc[-1:].drop('trial_id')#dfr[1].sort_values(by='mean_loss', ascending=False).reset_index().iloc[0]

    except:
        #row= dfr[1].iloc[-30:].mean() #dfr[1].sort_values(by='mean_loss', ascending=False).reset_index().iloc[0]
        row= dfr[1].sort_values(by=['mean_loss','training_iteration'], ascending=[True,False]).reset_index().iloc[-1:]

    find_name='train_set_TS_'
    index=dfr[0].find(find_name)
    loc = dfr[0]
    trial_id = dfr[0][index+len(find_name):index+len(find_name)+11]
    row['trial_id']=trial_id
    row['loc'] = loc
    if type(df) != pd.core.frame.DataFrame:
        df = pd.DataFrame(row)
        print(df)
    else:
        df=df.append(row,ignore_index=True)

locs=[]
for row in df.iterrows(): 
    locs.append(row[1]['loc']+'/checkpoint_'+str(row[1]['training_iteration']+1000000)[1:]+'/model.h5')
locs

#analysis = ExperimentAnalysis("/project/DSone/jaj4zcf/UCI_Time_Series/GridSearch2/gridsearchTwoPatterns/experiment_state-2022-04-05_07-53-36.json")
#analysis = ExperimentAnalysis("/project/DSone/jaj4zcf/UCI_Time_Series/GridSearch8/gridsearchArrowHead/experiment_state-2022-04-13_16-59-59.json")
from ray.tune import ExperimentAnalysis
#analysis = ExperimentAnalysis("/project/DSone/jaj4zcf/UCI_Time_Series/GridSearch2/gridsearchTwoPatterns/experiment_state-2022-04-05_07-53-36.json")
#analysis = ExperimentAnalysis("/project/DSone/jaj4zcf/UCI_Time_Series/GridSearch8/gridsearchArrowHead/experiment_state-2022-04-14_12-45-56.json")
loc_sort_list=list(np.flip(np.argsort(df.mean_loss)))
train_class=train_set_TS(config)
num_models = 20
failures=0
p=0
tune_loop_time = time.time() - start_time

locs=[]
for row in df.iterrows(): 
    locs.append(row[1]['loc']+'/checkpoint_'+str(row[1]['training_iteration']+1000000)[1:]+'/model.h5')
locs

loc_sort_list=list(np.flip(np.argsort(df.mean_loss)))
train_class=train_set_TS(config)
model=train_class.build_model()

model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(),
                          metrics=['accuracy'])
train_class.model = model

num_models = 20
failures=0
p=0
for i in loc_sort_list:
    loc = locs[i]
    try:
        train_class.model.load_weights(loc)
    except:
        print('no_model')
        failures = failures+1
        print(loc)
        continue
    if p>num_models-1:
        break
    if p==0:
        p=p+1
        pred=train_class.model.predict(train_class.x_test)
        votes=np.zeros_like(pred)
        vote = np.argmax(pred,axis=1)
        for i in range(0,vote.shape[0]):
            votes[i,vote[i]] = 1
    else:
        p=p+1
        pred_= train_class.model.predict(train_class.x_test)
        pred=pred+pred_
        votes_=np.zeros_like(pred)
        vote = np.argmax(pred_,axis=1)
        for i in range(0,vote.shape[0]):
            votes_[i,vote[i]] = 1
        votes=votes+votes_
    out=calculate_metrics(train_class.y_true, np.argmax(pred,axis=1))
    out2=calculate_metrics(train_class.y_true, np.argmax(votes,axis=1))
    mode='sum'
    write_out=config['dataset_name'] +',' +str(p)+','+ str(mode)+','+ str([tune_loop_time, out['precision'].values[0],out['accuracy'].values[0],out['recall'].values[0]])[1:-1] + ','+str(failures) + ','+str(sys.argv[10]) +str(sys.argv[8]) + '\n'
    f = open("/project/DSone/jaj4zcf/UCI_Time_Series/"+str(sys.argv[2])+"/"+"results.txt", "a")
    f.write(write_out)
    f.close()
    mode='vote'
    write_out=config['dataset_name'] +',' +str(p) +','+ str(mode)+','+ str([tune_loop_time, out2['precision'].values[0],out2['accuracy'].values[0],out2['recall'].values[0]])[1:-1] + ','+str(failures) + ','+str(sys.argv[10]) +str(sys.argv[8]) + '\n'
    f = open("/project/DSone/jaj4zcf/UCI_Time_Series/"+str(sys.argv[2])+"/"+"results.txt", "a")
    f.write(write_out)
    f.close()
    print(out)
    print(out2)
    print(failures)

2022-05-16 08:57:16,144	INFO experiment_analysis.py:749 -- No `self.trials`. Drawing logdirs from checkpoint file. This may result in some information that is out of sync, as checkpointing is periodic.


     index  mean_loss  mean_loss_avg  test_acc  train_acc        lr  \
312    309  -0.000806      -0.000806  0.931298        1.0  0.000132   

     best_loss  loss_count   done  timesteps_total  ...  time_this_iter_s  \
312   0.167338          14  False              NaN  ...         15.588837   

     time_total_s    pid     hostname         node_ip time_since_restore  \
312   4792.785695  26307  udc-ba25-23  10.155.195.126         155.891701   

     timesteps_since_restore  iterations_since_restore  warmup_time  \
312                        0                        10   172.821482   

                                                   loc  
312  /project/DSone/jaj4zcf/UCI_Time_Series/multiva...  

[1 rows x 27 columns]
{'dataset_name': 'EigenWorms', 'data_augment': True, 'monitor': True, 'model': 'inceptionFCNMulti2', 'depth': 4, 'filters': 16, 'dropout': 0, 'num_kernel_sizes': 3, 'dense_layer': 0, 'kernel_size': 41, 'Pert_Interval': 20, 'optimizer': 'ADAM', 'lr': 0.001, 'batch_size'

2022-05-16 08:59:47,340	INFO trainable.py:156 -- Trainable.setup took 150.223 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


{'dataset_name': 'EigenWorms', 'data_augment': True, 'monitor': True, 'model': 'inceptionFCNMulti2', 'depth': 4, 'filters': 16, 'dropout': 0, 'num_kernel_sizes': 3, 'dense_layer': 0, 'kernel_size': 41, 'Pert_Interval': 20, 'optimizer': 'ADAM', 'lr': 0.001, 'batch_size': 16, 'mini_batch_split': 10, 'patience': 75, 'split': 0, 'n_splits': 3, 'grid_search': False, 'epochs': 1, 'samples': 20, 'training_iterations': 1500, 'check_freq': 1, 'warmup': 50, 'use_val': False, 'initial_model_locs': [], 'grid_search_space': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'aug_noise_prob': <ray.tune.sample.Categorical object at 0x7fd41a6abb10>, 'aug_rotation_prob': <ray.tune.sample.Categorical object at 0x7fd41a6ab490>, 'aug_permutation_prob': <ray.tune.sample.Categorical object at 0x7fd41a6abc10>, 'aug_magnitude_prob': <ray.tune.sample.Categorical object at 0x7fd41a6ab110>, 'aug_scaling_prob': <ray.tune.sample.Categorical object at 0x7fd41a6ab910>, 'aug_time_warp_prob': <ray.tune.sample.Categorica

2022-05-16 09:02:18,825	INFO trainable.py:156 -- Trainable.setup took 151.438 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


24
24
24
24
24
24
48
Model Initialized
   precision  accuracy    recall  duration
0   0.925594  0.938931  0.911382       0.0
   precision  accuracy    recall  duration
0   0.925594  0.938931  0.911382       0.0
0
   precision  accuracy    recall  duration
0   0.945291  0.954198  0.942151       0.0
   precision  accuracy    recall  duration
0   0.925594  0.938931  0.911382       0.0
0
   precision  accuracy    recall  duration
0   0.945291  0.954198  0.942151       0.0
   precision  accuracy    recall  duration
0   0.925594  0.938931  0.911382       0.0
0
   precision  accuracy    recall  duration
0   0.909497  0.931298  0.909375       0.0
   precision  accuracy    recall  duration
0    0.91569  0.931298  0.895997       0.0
0
   precision  accuracy    recall  duration
0   0.909497  0.931298  0.909375       0.0
   precision  accuracy    recall  duration
0   0.909497  0.931298  0.909375       0.0
0
   precision  accuracy    recall  duration
0   0.909497  0.931298  0.909375       0.0
   pr